In [15]:
# this notebook creates and populates tables for UN Data reports

In [1]:
from functions import *
from SQL_functions import *
from stats_forumulas import *
import config
import mysql.connector
from mysql.connector import errorcode
import math
import pandas as pd
import numpy as np
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [4]:
cur, cnx = connect_to_db('gender')

## Creating Tables for UN Data

In [11]:
TABLES = {}
csv_names = ['child_birth_rate', 'contraceptive_prevalence', 'gender_urban_ratio', 
             'gender_rural_ratio', 'ter_ed_enrollment', 'women_teach_sec', 'women_teach_ter', 
             'abortion_laws', 'abortion_rate', 'abortion_info', 'population_data_full']

In [12]:
for name in csv_names:
    TABLES[name] = (f"""
    CREATE TABLE {name} (
    country varchar(50),
    subgroup varchar(200),
    year int(50),
    unit varchar(50),
    value int(50),
    PRIMARY KEY (country, subgroup, year)
    ) 
    """)

In [14]:
# added to DB
for name in csv_names:
    cur.execute(TABLES[name])

In [15]:
# close cursor
cnx.close()
cur.close() # returned True

True

## Populating our Tables

In [6]:
df.head()

,Last Updated Date,2019-10-02,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Country Name,Country Code,Indicator Name,Indicator Code,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,...,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,NaN
2,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,...,101669.0,102046.0,102560.0,103159.0,103774.0,104341.0,104872.0,105366.0,105845.0,NaN
3,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996973.0,9169410.0,9351441.0,9543205.0,9744781.0,9956320.0,...,29185507.0,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,NaN
4,Angola,AGO,"Population, total",SP.POP.TOTL,5454933.0,5531472.0,5608539.0,5679458.0,5735044.0,5770570.0,...,23356246.0,24220661.0,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0,29816748.0,30809762.0,NaN


In [7]:
# insert_data('population_data_full', 'population_data_full')


# cur, cnx = connect_to_db('gender')
df = pd.read_csv(f'Data To Use/population_data_full.csv', header = 4)
# df['2018'] = pd.to_numeric(df['2018'], errors = 'coerce')
print(df.head())
l = list(zip(df['Country Name'], list(range(df.shape[0])), list(range(df.shape[0])),
             list(range(df.shape[0])), df['2018']))
# # cleaning df to ensure proper end point
# start = df.loc[df['Data Source'] == 'Country Name'].index[0]
# df2 = df.iloc[start+1:]
qu = ''' INSERT INTO gender.population_data_full (country, subgroup, year, unit, value)
VALUES (%s, %s, %s, %s, %s)
'''
# # cleaning df to ensure proper type
# df2['Value'] = pd.to_numeric(df2['Value'], errors = 'coerce')
for item in l:
    cur.execute(qu, item)
cnx.commit()
cnx.close()

  Country Name Country Code     Indicator Name Indicator Code       1960  \
0        Aruba          ABW  Population, total    SP.POP.TOTL    54211.0   
1  Afghanistan          AFG  Population, total    SP.POP.TOTL  8996973.0   
2       Angola          AGO  Population, total    SP.POP.TOTL  5454933.0   
3      Albania          ALB  Population, total    SP.POP.TOTL  1608800.0   
4      Andorra          AND  Population, total    SP.POP.TOTL    13411.0   

        1961       1962       1963       1964       1965  ...      2010  \
0    55438.0    56225.0    56695.0    57032.0    57360.0  ...    101669   
1  9169410.0  9351441.0  9543205.0  9744781.0  9956320.0  ...  29185507   
2  5531472.0  5608539.0  5679458.0  5735044.0  5770570.0  ...  23356246   
3  1659800.0  1711319.0  1762621.0  1814135.0  1864791.0  ...   2913021   
4    14375.0    15370.0    16412.0    17469.0    18549.0  ...     84449   

       2011      2012      2013      2014      2015      2016      2017  \
0    102046    10

('Nigeria', 170, 170, 170, 195874740)
('Nicaragua', 171, 171, 171, 6465513)
('Netherlands', 172, 172, 172, 17231017)
('Norway', 173, 173, 173, 5314336)
('Nepal', 174, 174, 174, 28087871)
('Nauru', 175, 175, 175, 12704)
('New Zealand', 176, 176, 176, 4885500)
('OECD members', 177, 177, 177, 1303529456)
('Oman', 178, 178, 178, 4829483)
('Other small states', 179, 179, 179, 30758989)
('Pakistan', 180, 180, 180, 212215030)
('Panama', 181, 181, 181, 4176873)
('Peru', 182, 182, 182, 31989256)
('Philippines', 183, 183, 183, 106651922)
('Palau', 184, 184, 184, 17907)
('Papua New Guinea', 185, 185, 185, 8606316)
('Poland', 186, 186, 186, 37978548)
('Pre-demographic dividend', 187, 187, 187, 919485393)
('Puerto Rico', 188, 188, 188, 3195153)
('Korea, Dem. People’s Rep.', 189, 189, 189, 25549819)
('Portugal', 190, 190, 190, 10281762)
('Paraguay', 191, 191, 191, 6956071)
('West Bank and Gaza', 192, 192, 192, 4569087)
('Pacific island small states', 193, 193, 193, 2457367)
('Post-demographic divide

In [4]:
l = list(zip(df['Country Name'], list(range(df.shape[0])), list(range(df.shape[0])),
             list(range(df.shape[0])), df['2018']))
l

KeyError: 'Country Name'

In [14]:
# # see SQL_functions.py for function and helper function definitions
for name in csv_names:
    if name != 'abortion_info':
        insert_data(name, name)
    else:
        insert_data_special(name, name)

/Users/rebeccarosen/Desktop/flatiron 2019/mod3/project2/gender-and-tech/SQL_functions.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df2['Year'] = pd.to_numeric(df2['Year'], errors = 'coerce')
/Users/rebeccarosen/Desktop/flatiron 2019/mod3/project2/gender-and-tech/SQL_functions.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df2['Value'] = pd.to_numeric(df2['Value'], errors = 'coerce')


In [21]:
cur, cnx = connect_to_db('gender')
df = pd.read_csv('Data To Use/Honeypot_Women_in_Tech.csv')
cur.executemany(insert_more_query('women_in_tech'), get_more_tuples(df))
cnx.commit()
cnx.close()